In [67]:
# class Cache:
#     def __init__(self, line_size, capacity, associativity):
#         self.line_size = line_size
#         self.capacity = capacity
#         self.associativity = associativity
#         self.cache = {}
#
#     def access(self, address):
#         # Calculate cache line for the address
#         line = address // self.line_size
#         if line in self.cache:
#             # Cache hit
#             return True
#         else:
#             # Cache miss
#             if len(self.cache) >= self.capacity // self.line_size:
#                 self.cache.pop(next(iter(self.cache)))  # Simulate LRU eviction
#             self.cache[line] = True
#             return False
#
# # Simulate for row-major and column-major accesses
# cache = Cache(line_size=64, capacity=1024, associativity=4)
# row_major_hits = 0
# column_major_hits = 0
#
# # Row-major
# for i in range(64):
#     for j in range(64):
#         address = i * 64 + j  # Assume 64-byte rows
#         if cache.access(address):
#             row_major_hits += 1
#
# # Column-major
# cache = Cache(line_size=64, capacity=1024, associativity=4)
# for j in range(64):
#     for i in range(64):
#         address = i * 64 + j
#         if cache.access(address):
#             column_major_hits += 1
#
# print("Row-major hit rate:", row_major_hits / (64 * 64))
# print("Column-major hit rate:", column_major_hits / (64 * 64))


In [68]:
# import numpy as np
#
# # Cache Configuration
# CACHE_SIZE = 10 * 1024 * 1024  # 18 MiB for L3 Cache
# BLOCK_SIZE = 64  # Updated to 64 bytes for cache line consistency
# ASSOCIATIVITY = 4  # Matching associativity
# NUM_BLOCKS = CACHE_SIZE // BLOCK_SIZE
# SET_COUNT = NUM_BLOCKS // ASSOCIATIVITY
#
# # Cache Initialization
# cache = [{} for _ in range(SET_COUNT)]  # LRU cache simulation for each set
#
# # Function to calculate index and tag
# def calculate_index_and_tag(address):
#     block_offset_bits = int(np.log2(BLOCK_SIZE))
#     index_bits = int(np.log2(SET_COUNT))
#
#     block_address = address >> block_offset_bits
#     index = block_address & ((1 << index_bits) - 1)
#     tag = block_address >> index_bits
#
#     return index, tag
#
# # Enhanced cache access to account for block-level fetching
# def cache_access(address):
#     # Align address to block boundary
#     block_address = address // BLOCK_SIZE
#     index, tag = calculate_index_and_tag(block_address * BLOCK_SIZE)
#     cache_set = cache[index]
#
#     # Cache hit: tag exists in the set
#     if tag in cache_set:
#         # Update LRU position
#         cache_set.pop(tag)
#         cache_set[tag] = True
#         return "hit"
#
#     # Cache miss: tag not in the set
#     # Evict if set is full
#     if len(cache_set) >= ASSOCIATIVITY:
#         cache_set.pop(next(iter(cache_set)))  # Evict least recently used (first key)
#
#     # Insert new tag into the set (fetch block into cache)
#     cache_set[tag] = True
#     return "miss"
#
#
# # Simulate accesses with correct address calculation
# def simulate_accesses(array_shape, element_size, pattern="row-major", base_address=0):
#     MISS_COUNT, HIT_COUNT = 0, 0
#
#     # Reset cache state
#     for c_set in cache:
#         c_set.clear()
#
#     # Simulate accesses
#     if pattern == "row-major":
#         for i in range(array_shape[0]):
#             for j in range(array_shape[1]):
#                 address = base_address + ((i * array_shape[1] + j) * element_size)
#                 if cache_access(address) == "hit":
#                     HIT_COUNT += 1
#                 else:
#                     MISS_COUNT += 1
#     elif pattern == "column-major":
#         for j in range(array_shape[1]):
#             for i in range(array_shape[0]):
#                 address = base_address + ((i * array_shape[1] + j) * element_size)
#                 if cache_access(address) == "hit":
#                     HIT_COUNT += 1
#                 else:
#                     MISS_COUNT += 1
#
#     total_accesses = array_shape[0] * array_shape[1]
#     miss_rate = MISS_COUNT / total_accesses
#     return miss_rate
#
#
# # Data types with sizes
# DATA_TYPES = {
#     "char": (512, 512, 1),
#     "short": (256, 256, 2),
#     "int": (128, 128, 4),
#     "long": (64, 64, 8),
# }
#
# results = []
#
# for dtype, (rows, cols, size) in DATA_TYPES.items():
#     # Align base addresses explicitly for differentiation
#     # base_address_row = (np.random.randint(0, 2**16 // BLOCK_SIZE) * BLOCK_SIZE) & ~(BLOCK_SIZE - 1)
#     # base_address_col = ((np.random.randint(0, 2**16 // BLOCK_SIZE) * BLOCK_SIZE) + BLOCK_SIZE // 2) & ~(BLOCK_SIZE - 1)
#     base_address_row = 0x0
#     base_address_col = 0x800000  # Different base address
#
#     # Simulate row-major access
#     row_miss_rate = simulate_accesses((rows, cols), size, pattern="row-major", base_address=base_address_row)
#
#     # Reset cache and simulate column-major access
#     for c_set in cache:
#         c_set.clear()
#     col_miss_rate = simulate_accesses((rows, cols), size, pattern="column-major", base_address=base_address_col)
#
#     results.append((dtype, row_miss_rate, col_miss_rate))
#
# # Output results
# print("Data Type Miss Rate Analysis")
# print("------------------------------------------------")
# print(f"{'Data Type':<10}{'Row Major':<15}{'Column Major':<15}")
# for dtype, row_miss, col_miss in results:
#     print(f"{dtype:<10}{row_miss * 100:>10.2f}%{col_miss * 100:>15.2f}%")


In [69]:
import numpy as np

class Cache:
    def __init__(self, BLOCK_SIZE, CACHE_SIZE, associativity):
        self.BLOCK_SIZE = BLOCK_SIZE
        self.associativity = associativity
        self.num_sets = CACHE_SIZE // (BLOCK_SIZE * associativity)

        # Initialize the cache data: valid bits and tags
        self.valid_bits = np.zeros((self.num_sets, associativity), dtype=np.int8)
        self.tags = np.zeros((self.num_sets, associativity), dtype=np.int32)
        self.lru = np.zeros((self.num_sets, associativity), dtype=np.int8)  # To track LRU

    def access(self, address):
        # Calculate the cache index and tag
        line = address // self.BLOCK_SIZE
        cache_index = line % self.num_sets
        tag = line // self.num_sets

        # Check if the tag is in the cache set
        for way in range(self.associativity):
            if self.valid_bits[cache_index, way] == 1 and self.tags[cache_index, way] == tag:
                # Cache hit
                self.update_lru(cache_index, way)
                return True

        # Cache miss
        self.replace_line(cache_index, tag)
        return False

    def update_lru(self, cache_index, way):
        # Update the LRU status for all ways in the set
        self.lru[cache_index] = np.where(self.lru[cache_index] > self.lru[cache_index, way],
                                         self.lru[cache_index] - 1,
                                         self.lru[cache_index])
        self.lru[cache_index, way] = self.associativity - 1

    def replace_line(self, cache_index, tag):
        # Find the least recently used (LRU) way to replace
        lru_way = np.argmin(self.lru[cache_index])
        self.tags[cache_index, lru_way] = tag
        self.valid_bits[cache_index, lru_way] = 1
        self.update_lru(cache_index, lru_way)

# Function to determine cache layout description
def describe_cache_layout(associativity, num_sets):
    if associativity == 1:
        return "Direct-Mapped Cache"
    elif num_sets == 1:
        return "Fully-Associative Cache"
    else:
        return f"{associativity}-Way Set-Associative Cache"

# Function to simulate cache accesses and calculate miss rate
def simulate_cache(array_size, element_size, base_address, BLOCK_SIZE, CACHE_SIZE, associativity):
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)

    row_major_hits, column_major_hits = 0, 0

    # Row-major access
    for i in range(array_size):
        for j in range(array_size):
            address = base_address + (i * array_size + j) * element_size
            if cache.access(address):
                row_major_hits += 1

    row_major_miss_rate = 1 - (row_major_hits / (array_size * array_size))

    # Reset cache for column-major test
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)

    # Column-major access
    for j in range(array_size):
        for i in range(array_size):
            address = base_address + (i * array_size + j) * element_size
            if cache.access(address):
                column_major_hits += 1

    column_major_miss_rate = 1 - (column_major_hits / (array_size * array_size))

    return row_major_miss_rate, column_major_miss_rate

def calculate_array_size(CACHE_SIZE, element_size):
    total_elements = CACHE_SIZE // element_size  # Total elements that fit in the cache
    array_size = int(total_elements**0.5)       # Determine the size of the square array
    return array_size

# Constants
BLOCK_SIZE = 64  # Bytes
CACHE_SIZE = 4096  # Bytes
L3_CACHE_SIZE = 18 * 1024 * 1024  # Bytes
associativity = 4

base_address = 0x8aa1000  # Start at this arbitrary address


print("Question 1:")
# Cache layout description
cache_layout = describe_cache_layout(associativity, CACHE_SIZE // (BLOCK_SIZE * associativity))
print(f"Cache Layout: {cache_layout}\n")

# Data types and configurations
data_types = [
    {"name": "char", "element_size": 1, "array_size": 64},
    {"name": "short", "element_size": 2, "array_size": calculate_array_size(L3_CACHE_SIZE, 2)},
    {"name": "int", "element_size": 4, "array_size": 32},
    {"name": "long", "element_size": 8, "array_size": calculate_array_size(L3_CACHE_SIZE, 8)},
]

# Simulate and report miss rates
# Header
print(f"{'Data Type':<12} | {'Element Size':<12} | {'Array Size':<15} | {'Row-Major Miss Rate':<22} | {'Column-Major Miss Rate':<22}")
print("-" * 90)

for data_type in data_types:
    row_miss, col_miss = simulate_cache(
        data_type["array_size"],
        data_type["element_size"],
        base_address,
        BLOCK_SIZE,
        CACHE_SIZE,
        associativity
    )
    array_size_str = f"{data_type['array_size']}x{data_type['array_size']}"
    print(f"{data_type['name']:<12} | {data_type['element_size']:<12} | {array_size_str:<15} | {row_miss * 100:>6.2f}%{'':<15} | {col_miss * 100:>6.2f}%{'':<15}")


Question 1:
Cache Layout: 4-Way Set-Associative Cache

Data Type    | Element Size | Array Size      | Row-Major Miss Rate    | Column-Major Miss Rate
------------------------------------------------------------------------------------------
char         | 1            | 64x64           |   1.56%                |   1.56%               


KeyboardInterrupt: 

In [63]:
import random

# Function to simulate random cache accesses and calculate miss rate
def simulate_random_access(data_type, element_size, base_address, BLOCK_SIZE, CACHE_SIZE, associativity, L3_CACHE_SIZE):
    cache = Cache(BLOCK_SIZE, CACHE_SIZE, associativity)
    random_hits = 0
    num_accesses = L3_CACHE_SIZE // element_size // 2  # Half the range of L3_CACHE_SIZE

    # Perform random accesses
    for _ in range(num_accesses):
        address = base_address + random.randint(0, L3_CACHE_SIZE // 2) * element_size
        if cache.access(address):
            random_hits += 1

    random_miss_rate = 1 - (random_hits / num_accesses)
    return random_miss_rate

# Header for random accesses
print("\nQuestion 2: Random Access Miss Rates")
print(f"{'Data Type':<12} | {'Element Size':<12} | {'Array Size':<15} | {'Random Access Miss Rate':<30}")
print("-" * 80)

for data_type in data_types:
    random_miss = simulate_random_access(
        data_type["name"],
        data_type["element_size"],
        base_address,
        BLOCK_SIZE,
        CACHE_SIZE,
        associativity,
        L3_CACHE_SIZE
    )
    array_size_str = f"{data_type['array_size']}x{data_type['array_size']}"
    print(f"{data_type['name']:<12} | {data_type['element_size']:<12} | {array_size_str:<15} | {random_miss * 100:>6.2f}%{'':<15}")



Question 2: Random Access Miss Rates
Data Type    | Element Size | Array Size      | Random Access Miss Rate       
--------------------------------------------------------------------------------
char         | 1            | 64x64           |  99.96%               
short        | 2            | 3072x3072       |  99.98%               
int          | 4            | 32x32           |  99.99%               
long         | 8            | 1536x1536       |  99.99%               
